<a href="https://colab.research.google.com/github/vilim-cro/SLM-graph-extractor/blob/main/SLM_graph_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j langchain-google-vertexai json-repair

In [ ]:
import os
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
from langchain_community.llms import HuggingFacePipeline, HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_core.runnables import RunnableSequence
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.1,
)

#llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
from typing import Optional, List
from pydantic import BaseModel, Field

class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    birth_year: float = Field(
        ..., description="Year the person was born"
    )

class Country(BaseModel):
    """Information about a country."""
    name: Optional[str] = Field(..., description="The name of the country")



class People(BaseModel):
    """Identifying information about all people in a text."""

    people: List[Person]

class Countries(BaseModel):
    """Identifying information about all countries in a text."""

    countries: List[Country]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

In [ ]:
query =  """
Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
She was, in 1906, the first woman to become a professor at the University of Paris.
"""

In [ ]:
print(prompt.format_prompt(query=query).to_string())

In [ ]:
from langchain.chains import LLMChain

chain = RunnableSequence(prompt | llm)
result = chain.invoke({"query": query})


In [ ]:
print(result)

In [ ]:
parsed_result = parser.parse(result)
print(parsed_result)